Summary: these are the files required for the Julia simulation:

hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
test_calls = CSV.File("../test/austin-data/austin_test_calls_v1.csv")|> DataFrame 

In [51]:
import csv
import collections
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns

### looks like hospitals does not need to be updated for grid size. It is used in (call2hosp_event!) so I think I want demand2Hospital.

In [27]:
hospitals = (np.genfromtxt("../Input_Data/austin_data/hospitals.csv", delimiter=",", dtype = str))
hospitals[0]

array(['stn1_min', 'stn2_min', 'stn3_min', 'stn4_min', 'stn5_min',
       'stn6_min', 'stn7_min', 'stn8_min', 'stn9_min', 'stn10_min',
       'stn11_min', 'stn12_min', 'stn13_min', 'stn14_min', 'stn15_min',
       'stn16_min', 'stn17_min', 'stn18_min', 'stn19_min', 'stn20_min',
       'stn21_min', 'stn22_min', 'stn23_min', 'stn24_min', 'stn25_min',
       'stn26_min', 'stn27_min', 'stn28_min', 'stn29_min', 'stn30_min',
       'stn31_min', 'stn32_min', 'stn33_min', 'stn34_min', 'stn35_min',
       'stn36_min', 'stn37_min', 'stn38_min', 'stn39_min', 'stn40_min',
       'stn41_min', 'stn42_min', 'stn43_min', 'stn44_min',
       'HOSPITAL_NAME', 'LONGITUDE', 'LATITUDE', 'OBJECTID_1'],
      dtype='<U52')

### This also does not depend on grid, although I would be damned if I knew what it did. (I cannot find if it is used in the simulation.  It does tell the station longitude and latitude. 

In [28]:
stations = (np.genfromtxt("../Input_Data/austin_data/stations.csv", delimiter=",", dtype = str))
stations[0]

array(['stn1_min', 'stn2_min', 'stn3_min', 'stn4_min', 'stn5_min',
       'stn6_min', 'stn7_min', 'stn8_min', 'stn9_min', 'stn10_min',
       'stn11_min', 'stn12_min', 'stn13_min', 'stn14_min', 'stn15_min',
       'stn16_min', 'stn17_min', 'stn18_min', 'stn19_min', 'stn20_min',
       'stn21_min', 'stn22_min', 'stn23_min', 'stn24_min', 'stn25_min',
       'stn26_min', 'stn27_min', 'stn28_min', 'stn29_min', 'stn30_min',
       'stn31_min', 'stn32_min', 'stn33_min', 'stn34_min', 'stn35_min',
       'stn36_min', 'stn37_min', 'stn38_min', 'stn39_min', 'stn40_min',
       'stn41_min', 'stn42_min', 'stn43_min', 'stn44_min', 'STATION_NAME',
       'LONGITUDE', 'LATITUDE', 'OBJECTID_1'], dtype='<U19')

In [35]:
stations[:,-4] 

array(['STATION_NAME', 'Demand 01', 'Demand 02  ', 'Demand 03  ',
       'Demand 04 ', 'Demand 05', 'Demand 06', 'Demand 07', 'Station 01',
       'Station 02', 'Station 03', 'Station 04', 'Station 05',
       'Station 06', 'Station 07', 'Station 08', 'Station 09',
       'Station 10', 'Station 11', 'Station 12', 'Station 13',
       'Station 14', 'Station 15', 'Station 16', 'Station 17',
       'Station 18', 'Station 19', 'Station 20', 'Station 21',
       'Station 22', 'Station 23', 'Station 24', 'Station 25',
       'Station 26', 'Station 27', 'Station 28', 'Station 29',
       'Station 30', 'Station 31', 'Station 32', 'Station 33',
       'Station 34', 'Station 35', 'Station 36', 'Station 37'],
      dtype='<U19')

### Full_WeekdayCalls - call per regions? I feel like this is for the LP

In [38]:
Full_WeekdayCalls = (np.genfromtxt("../Input_Data/austin_data/Full_WeekdayCalls.csv", delimiter=",", dtype = str))
Full_WeekdayCalls[0]

array(['year', 'month', 'day', 'hour', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
       '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40',
       '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51',
       '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62',
       '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73',
       '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84',
       '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95',
       '96', '97', '98', '99', '100', '101', '102', '103', '104', '105',
       '106', '107', '108', '109', '110', '111', '112', '113', '114',
       '115', '116', '117', '118', '119', '120', '121', '122', '123',
       '124', '125', '126', '127', '128', '129', '130', '131', '132',
       '133', '134', '135', '136', '137', '138', '139', '140'

### adjacent_nbhd this is a 1-0 matrix

In [52]:
adjacent_nbhd = (np.genfromtxt("../Input_Data/austin_data/adjacent_nbhd.csv", delimiter=",", dtype = str))
print(adjacent_nbhd.shape)
adjacent_nbhd[1]

(225, 225)


array(['1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0

### coverage- this one might be tricky

In [44]:
coverage_real = (np.genfromtxt("../Input_Data/austin_data/coverage_real.csv", delimiter=",", dtype = str))
print(coverage_real.shape)

(210, 44)


In [50]:
### austin_test_calls_v1- I just need to change station numbers I think. Although I can check the mins if they are used.

In [47]:
austin_test_calls_v1 = (np.genfromtxt("../Input_Data/austin_data/austin_test_calls_v1.csv", delimiter=",", dtype = str))
print(austin_test_calls_v1.shape)
austin_test_calls_v1[0]

(30149, 59)


array(['interarrival_seconds', 'neighborhood', 'dow', 'stn1_min',
       'stn2_min', 'stn3_min', 'stn4_min', 'stn5_min', 'stn6_min',
       'stn7_min', 'stn8_min', 'stn9_min', 'stn10_min', 'stn11_min',
       'stn12_min', 'stn13_min', 'stn14_min', 'stn15_min', 'stn16_min',
       'stn17_min', 'stn18_min', 'stn19_min', 'stn20_min', 'stn21_min',
       'stn22_min', 'stn23_min', 'stn24_min', 'stn25_min', 'stn26_min',
       'stn27_min', 'stn28_min', 'stn29_min', 'stn30_min', 'stn31_min',
       'stn32_min', 'stn33_min', 'stn34_min', 'stn35_min', 'stn36_min',
       'stn37_min', 'stn38_min', 'stn39_min', 'stn40_min', 'stn41_min',
       'stn42_min', 'stn43_min', 'stn44_min', 'hosp1_min', 'hosp2_min',
       'hosp3_min', 'hosp4_min', 'hosp5_min', 'hosp6_min', 'hosp7_min',
       'hosp8_min', 'hosp9_min', 'hosp10_min', 'hosp11_min', 'hosp12_min'],
      dtype='<U21')

In [1]:
austin_test_calls_v1[0]

NameError: name 'austin_test_calls_v1' is not defined